# Рекуррентные сети пишут тексты
__Суммарное количество баллов: 12__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW06] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит познакомиться с объединением Deep Learning и NLP. Для начала предстоит построить векторное пространство для словоря, а затем применить его для предсказания следующих слов в тексте.

In [1]:
import torch
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import random
import json
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.neighbors import KDTree

In [2]:
from collections import Counter

In [3]:
tokenized = json.load(open("opencorp.json", "r", encoding="UTF-8"))

In [7]:
for sentence in tokenized[:3]:
    sentence =  ['<START>'] + sentence + ['<END>']
    print(sentence)

['<START>', '«', 'школа', 'злословия', '»', 'учит', 'прикусить', 'язык', 'сохранится', 'ли', 'градус', 'дискуссии', 'в', 'новом', 'сезоне', '?', '<END>']
['<START>', 'великолепная', '«', 'школа', 'злословия', '»', 'вернулась', 'в', 'эфир', 'после', 'летних', 'каникул', 'в', 'новом', 'формате', '.', 'в', 'истории', 'программы', 'это', 'уже', 'не', 'первый', '«', 'ребрендинг', '»', '.', 'сейчас', 'с', 'трудом', 'можно', 'припомнить', ',', 'что', 'начиналась', '«', 'школа', '…', '»', 'на', 'канале', '«', 'культура', '»', 'как', 'стандартное', 'ток-шоу', ',', 'которое', 'отличалось', 'от', 'других', '«', 'кухонными', '»', 'обсуждениями', 'гостя', ',', 'что', 'называется', '–', '«', 'за', 'глаза', '»', ',', 'и', 'неожиданными', 'персонами', 'в', 'качестве', 'ведущих', '.', '<END>']
['<START>', 'писательница', 'татьяна', 'толстая', 'и', 'сценаристка', 'дуня', 'смирнова', 'вроде', 'бы', 'не', 'вполне', 'соответствовали', 'принятым', 'на', 'российском', 'телевидении', 'стандартам', 'телеведущи

### Задание 1 (3 балла)
Чтобы обучить нейронную сеть, нам нужен датасет. В данном задании предлагается использовать данные, полученные из корпуса текстов OpenCorpora. Более того, датасет нужно представить в удобном виде. Поскольку мы хотим обучать эмбеддинги на парах `(token_center, token_context)`, а также иметь возможность делать `negative sampling`, датасет должен уметь выдавать соответствующие пары, а так же `negative sampling`-токены. 

Кроме того, мы бы не хотели строить эмбеддинги для очень редких слов, поэтому в словарь и в пары должны входить только слова, которые встречаются более `count_threshold` раз, а остальные должны быть заменены на специальный токен `"<UNKNOWN>"`. Последовательность должна начинаться с токена `"<START>"` и заканчиваться токеном `"<END>"`.

#### Методы
`__init__` - принимает на вход список последовтельностей токенов, преобразуя в соответствии с описанными выше критериями. При инициализации списка токенов нужно учитывать, что с вероятностью $1 - (\sqrt{\frac{0.001}{f(t)}} + 1) \cdot \frac{f(t)}{0.001}$ ($f(t)$ - частота слова в корпусе) мы "выкидываем" слово из текста, не добавляя никакие пары токенов с его участием в список. Это нужно для того, чтобы мы не переобучались на часто встречаемые слова. Также в `self.voc` должен записать актуальный словарь токенов.

`__len__` - возвращает количество пар `(token_center, token_context)`

`__getitem__` - принимает на вход индекс `i`, соответствующий паре `(t_i, c_i)`. Возвращает пару тензоров `(t_i, [c_i] + negatives)`, где `negatives` - список негативных токенов.

`negative_sampling` - осуществляет взвешенное негативное сэмплирование. Вес токена определяется как $\frac{(count(t))^{0.75}}{\sum (count(t))^{0.75}}$, т.е. в negative samples частые слова попадают чаще, чем другие.

In [4]:
class TokenDataset(Dataset):
    def __init__(self, tokenized_sources, window=3, count_threshold=5, negative_sampling=5):
        self.token_pairs = []
        self.window = window
        self.count_threshold = count_threshold
        self.neg_sampling = negative_sampling
        self.voc, self.counts = self.make_dictionary(tokenized_sources)
        self.size = len(self.voc)
        
        #self.voc = set(["<START>", "<END>", "<UNKNOWN>"])
        #self.counts = {"<START>": len(tokenized_sources), "<END>": len(tokenized_sources), "<UNKNOWN>":1}
        
        self.idx2token = list(self.voc)
        self.token2idx = {token: i for i, token in enumerate(self.idx2token)}
        
        self.pair = self.make_pair(tokenized_sources)
        sum_count = sum(np.array(list(self.counts.values()))**0.75)
        self.negative_prob = [v**0.75/sum_count for v in self.counts.values()]
                    
    def negative_sampling(self):
        return np.random.choice(np.array(self.size), size=self.neg_sampling, replace=False, p=self.negative_prob).tolist()
    
    def make_dictionary(self, tokenized):
        dictionary = set()
        tokens = [token for sentence in tokenized for token in sentence]
        
        counter = dict(Counter(tokens))
        unknown = []
        for token, n in counter.items():
            if n > self.count_threshold:
                dictionary.add(token)
            else:
                unknown.append(token)
        for token in unknown:
            del counter[token]

                
        counter["<UNKNOWN>"] = len(unknown)
        counter["<START>"] = len(tokenized)
        counter["<END>"] = len(tokenized)
        dictionary = dictionary | set(["<START>", "<END>", "<UNKNOWN>"])
        return dictionary, counter
    
    def make_pair(self, tokenized):
        pair = []
        num = self.size
        
        probs = {item[0]: (np.sqrt((item[1]/num)/0.001) + 1)*(0.001/(item[1]/num)) for item in self.counts.items()}## вероятность оставить слово
        
        for sentence in tokenized:
            sentence =  ['<START>'] + sentence + ['<END>']
            new_sentence = [] ## записываю индекс токена в словаре
            
            for center in sentence:
                if center in self.voc: 
                    if probs[center] < np.random.uniform(0, 1) or center =='<START>' or center == '<END>':
                        new_sentence.append(self.token2idx[center])

            
            for center in range(len(new_sentence)):
                
                l = max(0, center - self.window)
                r = min(len(new_sentence) - 1, center + self.window)
                
                for i in range(l, center):
                    pair.append((new_sentence[center],new_sentence[i]))
                
                for i in range(center + 1, r):
                    pair.append((new_sentence[center],new_sentence[i]))
                    
        return pair                      
        
    def __getitem__(self, index):
        elem1, elem2 = self.pair[index]
        return torch.tensor([elem1], dtype=torch.long), torch.tensor([elem2] + self.negative_sampling(), dtype=torch.long)

    def __len__(self):
        return len(self.token_pairs)



In [5]:
t = TokenDataset(tokenized)

### Задание 2 (4 балла)
Теперь реализуем непосредственно SkipGram. Для этого нам потребуются `torch.autograd.Variables` чтобы сделать эмбеддинги обучаемыми. Также сразу реализуем интерфейс, которым будем пользоваться для применения эмбеддингов в следующих задачах.

#### Методы SkipGram
`__init__` - принимает на вход словарь и размерность пространств эмбеддингов. Инициализирует эмбеддинги для центрального и контекстного слов.

`get_variables` - возвращает лист из всех `torch.autograd.Variables`. Необходимо, чтобы инициализировать оптимизатор.

`predict_proba(center_tokens, context_tokens)` - принимает на вход список центральных токенов и список списков токенов из предполагаемого контекста. Для каждого центрального токена и соответствующего ему списка контекстных токенов должен вернуть скалярное произведение центрального и контекстуального эмбеддингов.

#### Методы Embedding
`__init__` - принимает на вход обученный SkipGram

`embed` - возвращает эмбеддинги для всех элементов списка

`reconstruct` - для всех элементов списка возвращает наиболее подходящий токен. Не возвращает `"<UNKNOWN>"`

`n_closest` - возвращает `n` ближайших токенов для каждого элемента списка. Не возвращает `"<UNKNOWN>"`

In [6]:
class SkipGram:
    def __init__(self, voc, latent_size):
        self.id2token = list(voc)
        self.token2id = dict([(token, i) for i, token in enumerate(self.id2token)])
        self.latent_size = latent_size
        self.W_center = torch.randn(len(voc), self.latent_size, requires_grad=True)
        self.W_context = torch.randn(self.latent_size, len(voc), requires_grad=True)
        
    def get_variables(self):
        return [self.W_center, self.W_context]
    
    def predict_proba(self, center_ids, context_ids):
        center = self.W_center[center_ids]
        context = self.W_context[:, context_ids].transpose(1, 0)
        return torch.bmm(center, context).reshape(context_ids.size())
    
class Embedding:
    def __init__(self, skip_gram):
        self.skip_gram = skip_gram
        self.W_center = skip_gram.W_center.detach().numpy()
        self.W_context = skip_gram.W_context.transpose(1, 0).detach().numpy()
        self.tree = KDTree(matrix, leaf_size=1, metric="euclidean")
    
    def embed(self, tokens):
        ind = self.skip_gram.token2id[tokens]
        return self.W_center[ind] + self.W_context[ind]
    
    def reconstruct(self, embeddings):
        return n_closest(self, embeddings, n=1)

    def n_closest(self, embeddings, n=5):
        neighbors = self.tree.quary(embeddings, k = n+1, return_distance=False)[0][1:]
        closest = []
        i=0
        while i<n:
            neighbor = self.skip_gram.id2token[neighbors[i]]
            if neighbor != '<UNKNOWN>':
                closest.append(neighbor)
            i+=1
        return closest

In [7]:
BATCH_SIZE = 4096
EPOCHS = 200

print("Creating dataset...")
dataset = TokenDataset(tokenized, count_threshold=10, window=4, negative_sampling=10)
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True)
print("Creating skipgram...")
skipgram = SkipGram(dataset.voc, 150)
optim = torch.optim.Adam(skipgram.get_variables(), lr=1e-3)
y = torch.tensor([0] * (BATCH_SIZE + 1), dtype=torch.long).cuda()
losses = []

for i in range(EPOCHS):
    avg_loss = 0
    steps = 0
    for batch in tqdm(dataloader):
        probs = skipgram.predict_proba(batch[0], batch[1])
        loss = F.cross_entropy(probs, y[:len(probs)])
        optim.zero_grad()
        loss.backward()
        optim.step()
        avg_loss += loss.item()
        steps += 1
    losses.append(avg_loss/steps)
    print("Loss:", avg_loss/steps)

plt.figure(figsize=(12, 8))
plt.plot(list(range(EPOCHS)), losses)
plt.tight_layout()
plt.show()

Creating dataset...


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
embedder = Embedding(skipgram)

In [ ]:
king = embedder.embed(["король"])[0]
cat = embedder.embed(["кошка"])[0]
owl = embedder.embed(["сыч"])[0]
give = embedder.embed(["дать"])[0]
me = embedder.embed(["ты"])[0]
you = embedder.embed(["я"])[0]
print(embedder.n_closest([king], 10)[0])
print(embedder.n_closest([cat], 10)[0])
print(embedder.n_closest([owl], 10)[0])
print(embedder.n_closest([give], 10)[0])
print(embedder.n_closest([you], 10)[0])
print(embedder.n_closest([me], 10)[0])

### Задание 3 (2 балла)
Теперь будем учиться восстанавливать слова в тексте. Для этого нам потребуется также определить датасет последовательностей фиксированной длинны.

#### Методы
`__init__` - принимает на вход `embedder` (обученный SkipGram) и список токенизированных последоватлеьностей `tokenized`.

`__getitem__` - возвращает случайную закодированную при помощи SkipGram подпоследовательность длины `seq_len` одной из исходных последовательностей, сдвинутую на один токен подпоследовательность (т.е. следующие слова в тексте) и маску, которая отражает то, является ли токен неизвестным (`"<UNKNOWN"`).

`__len__` - равна количеству последовательностей.

In [ ]:
class TokenSeqDataset(Dataset):
    def __init__(self, embedder, tokenized, seq_len=32):
        self.seq_len = seq_len
            
    def __len__(self):
        return None

    def __getitem__(self, index):
        return None, None, None

### Задание 4 (2 балла)
Теперь обучим рекуррентную сеть, которая будет предсказывать следующее слово в тексте. Модель будет состоять из трех блоков: `input` (отвечает за предоброботку эмбеддинга), `rnn` (рекуррентная часть), `output` (отвечает за постобработку выхода).

#### Методы
`predict_sequential` - возвращает последовательность предсказаний для батча последовательностей

`get_next` - предсказывает следующее слово

`reset` - обнуляет внутреннее состояние сети

In [ ]:
class TextRNN:
    def __init__(self, latent_space=150, hidden_layer=512):
        self.input = torch.nn.Sequential(
            torch.nn.Linear(latent_space, hidden_layer),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layer, hidden_layer),
            torch.nn.ReLU()
        )
        self.rnn = None
        self.output = None
        self.input.cuda()
        self.rnn.cuda()
        self.output.cuda()
        self.hidden = None
    
    def predict_sequential(self, sequences):
        x, _ = self.rnn(self.input(sequences))
        return self.output(x)
    
    def parameters(self):
        return list(self.input.parameters()) + list(self.rnn.parameters()) + list(self.output.parameters())
    
    def get_next(self, batch):
        x, self.hidden = self.rnn(self.input(sequences).unsqueeze(1))
        return self.output(x)
    
    def reset(self):
        self.hidden = None

In [ ]:
BATCH_SIZE = 64
EPOCHS = 200

dataset = TokenSeqDataset(embedder, tokenized)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
rnn = TextRNN(150)
optim = torch.optim.Adam(rnn.parameters(), lr=1e-3)
losses = []
top1accs = []
top5accs = []
for i in range(EPOCHS):
    avg_loss = 0
    top1acc = 0
    top5acc = 0
    steps = 0
    acc_steps = 0
    for x, y_true, loss_mask in tqdm(dataloader):
        y_pred = rnn.predict_sequential(x.cuda())
        loss = (((y_true.cuda() - y_pred)**2).mean(dim=-1) * loss_mask.cuda()).mean()
        optim.zero_grad()
        loss.backward()
        optim.step()
        avg_loss += loss.item()
        steps += 1
        if steps % 100 == 0:
            acc_steps += 1
            word_pred = embedder.n_closest(y_pred.detach().view(-1, 150)[:200].cpu().numpy())
            word_true = embedder.reconstruct(y_true.detach().view(-1, 150)[:200].cpu().numpy())
            unknown_mask = loss_mask.view(-1).cpu().numpy()
            t1a = 0
            t5a = 0
            for true, pred, is_unknown in zip(word_true, word_pred, loss_mask):
                if is_unknown:
                    continue
                if true == pred[0]:
                    t1a += 1
                if true in pred:
                    t5a += 1
            top1acc += t1a / len(word_pred)
            top5acc += t5a / len(word_pred)
    losses.append(avg_loss/steps)
    top1accs.append(top1acc/acc_steps)
    top5accs.append(top5acc/acc_steps)
    print("Loss:", avg_loss/steps)
    print("Top-1 accuracy:", top1acc/acc_steps)
    print("Top-5 accuracy:", top5acc/acc_steps)

plt.figure(figsize=(12, 8))
plt.plot(list(range(EPOCHS)), losses)
plt.tight_layout()
plt.show()
plt.figure(figsize=(12, 8))
plt.plot(list(range(100)), top1accs, label="Top-1")
plt.plot(list(range(100)), top5accs, label="Top-5")
plt.legend()
plt.tight_layout()
plt.show()

### Задание 5 (1 балл)
Отлично, осталось только научитсья итеративно продолжать последовательность. Давайте попробуем научиться это делать.

#### Методы
`continue_sequence` - возвращает завершенную последовательность. Входная последовательность может быть пустой, поэтому в начало нужно добавить токен `"<START>"`. Закончить построение последовательности нужно после получения токена `"<END>"` или после получения `max_len` новых слов.

In [ ]:
class SequenceCompleter:
    def __init__(self, rnn, embedder, max_len=128):
        self.rnn = rnn
        self.embedder = embeder
        self.max_len = max_len
        
    def continue_sequence(self, sequence):
        t_sequence = ["<START>"] + sequence
        embedding = self.embedder.embed(t_sequence)
        self.rnn.reset()
        with torch.no_grad():
            for e in embedding:
                x = self.rnn.get_next(torch.tensor([e], dtype=torch.float).cuda())
            rec = self.embeder.reconstruct(x)
            continued_sequence = []
            ctn = 0
            while rec[0] != "<END>" and ctn < self.max_len:
                continued_sequence.append(rec[0])
                e = self.embedder.embed(rec)
                x = self.rnn.get_next(torch.tensor(e, dtype=torch.float).cuda())
                rec = self.embeder.reconstruct(x)
        return sequence + continued_sequence

In [ ]:
seq_completer = SequenceCompleter(rnn, embedder)
print(seq_completer.continue_sequence(["учеба", "в", "магистратуре", "-", "это"]))
print(seq_completer.continue_sequence(["работает", "ли", "наша", "простая", "модель", "?"]))
print(seq_completer.continue_sequence(["я", "точно", "знаю"]))
print(seq_completer.continue_sequence(["машина", "времени"]))
print(seq_completer.continue_sequence(["сегодня"]))
print(seq_completer.continue_sequence([]))

In [ ]:
class CharLSTM:
    def __init__(self, symbols):
        self.hidden = None
        self.input = torch.nn.Linear(len(symbols), 128)
        self.lstm = torch.nn.LSTM(128, 128)
        self.output = torch.nn.Sequential(torch.nn.Linear(128, 128), torch.nn.ReLU(), torch.nn.Linear(128, len(symbols)))
    
    def loss(self, batch):
        pass